In [5]:
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer,OneHotEncoder, RobustScaler
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import joblib
import pickle
import mlflow
import mlflow.sklearn

In [6]:
df=pd.read_csv('data/data.csv')
df=df.dropna(subset=['SiteEnergyUse(kBtu)'])

In [7]:
def replace_building_type(df):
    df['BuildingType'] = df['BuildingType'].replace({'Nonresidential COS': 'Nonresidential', 'Nonresidential WA': 'Nonresidential'})
    return df


In [1]:
supprimer= [
    'Address',
    'City',
    'Comments',
    'ComplianceStatus',
    'CouncilDistrictCode',
    'DataYear',
    'DefaultData',
    'Electricity(kBtu)',
    'Electricity(kWh)',
    'ENERGYSTARScore',
    'GHGEmissionsIntensity',
    'LargestPropertyUseType',
    'LargestPropertyUseTypeGFA',
    'Latitude',
    'ListOfAllPropertyUseTypes',
    'Longitude',
    'NaturalGas(kBtu)',
    'NaturalGas(therms)',
    'Neighborhood',
    'OSEBuildingID',
    'Outlier',
    'PropertyGFAParking',
    'PropertyGFATotal',
    'PropertyName',
    'SecondLargestPropertyUseType',
    'SecondLargestPropertyUseTypeGFA',
    'SiteEnergyUseWN(kBtu)',
    'SiteEUI(kBtu/sf)',
    'SiteEUIWN(kBtu/sf)',
    'SourceEUI(kBtu/sf)',
    'SourceEUIWN(kBtu/sf)',
    'State',
    'SteamUse(kBtu)',
    'TaxParcelIdentificationNumber',
    'ThirdLargestPropertyUseType',
    'ThirdLargestPropertyUseTypeGFA',
    'TotalGHGEmissions',
    'YearsENERGYSTARCertified',
    'ZipCode'
]

thresholds = [1.0e6, 2.5e6, 5.0e6, 1.0e7]

In [3]:
df = df.drop(supprimer, axis=1, inplace=True)

NameError: name 'df' is not defined

In [4]:
df

NameError: name 'df' is not defined

In [5]:

categorical_features=['BuildingType', 'PrimaryPropertyType']
numeric_features=['YearBuilt', 'NumberofBuildings','NumberofFloors', 'PropertyGFABuilding(s)']

In [6]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=1)),
    ('scaler', RobustScaler())
])
categorical_transformer = Pipeline(steps=[
    ('replace_building_type', FunctionTransformer(replace_building_type, validate=False)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop' 
)

# Création du pipeline final
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


In [7]:
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=1,
                                                                                 strategy='constant')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['YearBuilt',
                                                   'NumberofBuildings',
                                                   'NumberofFloors',
                                                   'PropertyGFABuilding(s)']),
                                                 ('cat',
                                                  Pipeline(steps=[('replace_building_type',
                                                                   FunctionTransformer(func=<function replace_building_type at 0x7f417ad72ef0>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BuildingType',
                                                   'PrimaryPropertyType'])])),
                ('regressor', LinearRegression())])

In [8]:
# Définir la target et les features après preprocessing
y = df['SiteEnergyUse(kBtu)'].astype(float)
X = df.drop(columns=['SiteEnergyUse(kBtu)'])


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Entraîner le modèle et prédire
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Calculer le coefficient de détermination R^2
r2 = r2_score(y_test, y_pred)
mae=mean_absolute_error(y_test, y_pred)

print(f"Coefficient de détermination R^2 : {r2:.4f}")
print(f"MAE : {mae:.4f}")
model_path = 'trained_model.pkl'

# Exportation du modèle avec pickle
with open(model_path, 'wb') as f:
    pickle.dump(pipeline, f)

print(f"Modèle exporté avec succès dans : {model_path}")

Coefficient de détermination R^2 : 0.8274
MAE : 3188614.2721
Modèle exporté avec succès dans : trained_model.pkl


In [9]:
# Définir la target et les features après preprocessing
y = df['SiteEnergyUse(kBtu)'].astype(float)
X = df.drop(columns=['SiteEnergyUse(kBtu)'])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Démarrer un run dans MLflow
with mlflow.start_run():

    # Entraîner le modèle et prédire
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    # Calculer le coefficient de détermination R^2
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)

    # Log des paramètres et des métriques
    mlflow.log_param("model_type", "XGBoost")  # Exemple, vous pouvez ajouter plus de paramètres
    mlflow.log_metric("R2_score", r2)
    mlflow.log_metric("MAE", mae)

    # Enregistrer le modèle dans MLflow
    mlflow.sklearn.log_model(pipeline, "model")

    # Exportation du modèle avec pickle
    model_path = 'trained_model.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump(pipeline, f)

    # Log de l'artefact (modèle exporté)
    mlflow.log_artifact(model_path)

    print(f"Modèle exporté avec succès dans : {model_path}")
    print(f"R^2 : {r2:.4f}")
    print(f"MAE : {mae:.4f}")


2025/03/11 16:00:56 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.


Modèle exporté avec succès dans : trained_model.pkl
R^2 : 0.8274
MAE : 3188614.2721


In [10]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV

# Démarrer MLflow Tracking
mlflow.set_experiment("Regression_Models")

# Models to test
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet()
}

# Parameter grids for hyperparameter tuning
param_grids = {
    'Ridge': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'Lasso': {'regressor__alpha': [0.1, 1.0, 10.0]},
    'ElasticNet': {
        'regressor__alpha': [0.1, 1.0, 10.0],
        'regressor__l1_ratio': [0.25, 0.5, 0.75]
    }
}

# Create an empty dictionary to store results
results = {}

for model_name, model in models.items():
    with mlflow.start_run(run_name=model_name):
        
        # Mise à jour du modèle dans le pipeline
        pipeline.steps[-1] = ('regressor', model)
        
        # Grid search pour optimiser les hyperparamètres
        grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=5, scoring='r2')
        grid_search.fit(X_train, y_train)
        
        best_model = grid_search.best_estimator_
        test_score = grid_search.score(X_test, y_test)
        
        # Stocker les résultats
        results[model_name] = {
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'test_score': test_score
        }
        
        # Enregistrement des paramètres et métriques dans MLflow
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metric("best_r2", grid_search.best_score_)
        mlflow.log_metric("test_r2", test_score)
        
        # Enregistrement du modèle entraîné
        mlflow.sklearn.log_model(best_model, f"{model_name}_model")

# Affichage des résultats
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Best cross-validation score: {result['best_score']:.4f}")
    print(f"Test score: {result['test_score']:.4f}")
    print(f"Best parameters: {result['best_params']}")
    print()

print("Les résultats sont enregistrés dans MLflow. Lancez 'mlflow ui' pour les visualiser.")


2025/03/11 16:01:13 INFO mlflow.tracking.fluent: Experiment with name 'Regression_Models' does not exist. Creating a new experiment.
/home/ndaoalassane/.pyenv/versions/lewagon/lib/python3.10/site-packages/_distutils_hack/__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ndaoalassane/.pyenv/versions/lewagon/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils

Model: Ridge
Best cross-validation score: 0.5677
Test score: 0.8499
Best parameters: {'regressor__alpha': 1.0}

Model: Lasso
Best cross-validation score: 0.5615
Test score: 0.8275
Best parameters: {'regressor__alpha': 10.0}

Model: ElasticNet
Best cross-validation score: 0.4837
Test score: 0.9275
Best parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.75}

Les résultats sont enregistrés dans MLflow. Lancez 'mlflow ui' pour les visualiser.


In [14]:
!mlflow ui

[2025-03-11 16:02:29 +0100] [322721] [INFO] Starting gunicorn 22.0.0
[2025-03-11 16:02:29 +0100] [322721] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-03-11 16:02:29 +0100] [322721] [ERROR] Retrying in 1 second.
[2025-03-11 16:02:30 +0100] [322721] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-03-11 16:02:30 +0100] [322721] [ERROR] Retrying in 1 second.
[2025-03-11 16:02:31 +0100] [322721] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-03-11 16:02:31 +0100] [322721] [ERROR] Retrying in 1 second.
[2025-03-11 16:02:32 +0100] [322721] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-03-11 16:02:32 +0100] [322721] [ERROR] Retrying in 1 second.
[2025-03-11 16:02:33 +0100] [322721] [ERROR] Connection in use: ('127.0.0.1', 5000)
[2025-03-11 16:02:33 +0100] [322721] [ERROR] Retrying in 1 second.
[2025-03-11 16:02:34 +0100] [322721] [ERROR] Can't connect to ('127.0.0.1', 5000)
Running the mlflow server failed. Please see the logs above for details.


In [12]:
# Utiliser un modèle de forêt aléatoire pour la sélection des caractéristiques
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# Utiliser SelectFromModel pour sélectionner les caractéristiques importantes
selector = SelectFromModel(model, prefit=True)
X_selected = selector.transform(X)

# Vérifier les caractéristiques sélectionnées
selected_features = poly_features.columns[selector.get_support()]
print(selected_features)

ValueError: could not convert string to float: 'NonResidential'